In [ ]:
%matplotlib inline


PyTorch: nn
-----------

A fully-connected ReLU network with one hidden layer, trained to predict y from x
by minimizing squared Euclidean distance.

This implementation uses the nn package from PyTorch to build the network.
PyTorch autograd makes it easy to define computational graphs and take gradients,
but raw autograd can be a bit too low-level for defining complex neural networks;
this is where the nn package can help. The nn package defines a set of Modules,
which you can think of as a neural network layer that has produces output from
input and may have some trainable weights.

This is a modified version of the original PyTorch example at https://github.com/pytorch/tutorials/blob/master/beginner_source/examples_nn/two_layer_net_nn.py



In [ ]:
import torch
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.

# Step 1
N, D_in, H, D_out = 64, 1000, 100, 10

# Step 2
# N, D_in, H, D_out = 64, 1000, 5000, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

display(x, x.shape)
display(y, y.shape)


In [ ]:
# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    
    # Step 2
    # torch.nn.Linear(H, H),
    # torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

# Step 1
learning_rate = 1e-4

# Step 2
# learning_rate = 1e-5

pass_data = []
loss_data = []
pass_data_set = []
loss_data_set = []

for t in range(1000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    pass_data.append(t)
    loss_data.append(loss.item())
    
    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

In [ ]:
display(pass_data)
display(loss_data)

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, model):
        self.loss_data = loss_data

    def hovertool(self):
        tooltips = [
            ("LossValue", "@Loss"),
            ("Pass", "@Pass"),
        ]

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(loss_data)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='linear')
#p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')

chart_data = {'Pass': pass_data, 'Loss': loss_data}
p.line(source=chart_data, x='Pass', y='Loss')
show(p)